<a href="https://colab.research.google.com/github/markusloecher/DataScience2021/blob/main/TWSM/Similarity_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Similarity Functions

#### TOKEN-BASED SIMILARITY

Token-based similarity measures compare two strings by first dividing them into a set of tokens
using a tokenization function, which we denote as tokenize(·). Intuitively, tokens correspond to
substrings of the original string. As a simple example, assume the tokenization function splits a
string into tokens based on whitespace characters.Then, the string Sean Connery results in the set
of tokens *{Sean,Connery}*. As we will show throughout our discussion, the main advantage of
token-based similarity measures is that the similarity is less sensitive to word swaps compared to
similarity measures that consider a string as a whole (notably edit-based measures). That is, the
comparison of *Sean Connery* and *Connery Sean* will yield a maximum similarity score because both
strings contain the exact same tokens. On the other hand, typographical errors within tokens are
penalized, for instance, the similarity of *Sean Connery* and *Shawn Conery* will be zero.

#### JACCARD COEFFICIENT

The Jaccard coefficient is a similarity measure that, in its most general form, compares two sets P
and Q with the following formula:
$$Jaccard(P,Q) = \frac{|P \cap Q|}{|P \cup Q|}$$
Essentially,the Jaccard coefficient measures the fraction of the data that is shared between P
and Q, compared to all data available in the union of these two sets.

An advantage of the Jaccard coefficient is that it is not sensitive to word swaps. Indeed, the
score of two names *John Smith* and *Smith John* would correspond to the score of exactly equal strings because the Jaccard coefficient considers only whether a token exists in a string, not at which position.

In [ ]:
import nltk
 
sent1 = set("It might help to re-install Python if possible.")
sent2 = set("It can help to install Python again if possible.")
sent3 = set("It can be so helpful to reinstall C++ if possible.")
sent4 = set("help It possible Python to re-install if might.") # This has the same words as sent1 with a different order.
sent5 = set("I love Python programming.")
 
jd_sent_1_2 = nltk.jaccard_distance(sent1, sent2)
jd_sent_1_3 = nltk.jaccard_distance(sent1, sent3)
jd_sent_1_4 = nltk.jaccard_distance(sent1, sent4)
jd_sent_1_5 = nltk.jaccard_distance(sent1, sent5)
 
 
print(jd_sent_1_2, 'Jaccard Distance between sent1 and sent2')
print(jd_sent_1_3, 'Jaccard Distance between sent1 and sent3')
print(jd_sent_1_4, 'Jaccard Distance between sent1 and sent4')
print(jd_sent_1_5, 'Jaccard Distance between sent1 and sent5')

0.18181818181818182 Jaccard Distance between sent1 and sent2
0.36 Jaccard Distance between sent1 and sent3
0.0 Jaccard Distance between sent1 and sent4
0.22727272727272727 Jaccard Distance between sent1 and sent5


#### COSINE SIMILARITY USINGTOKEN FREQUENCY AND INVERSE DOCUMENT FREQUENCY

The cosine similarity is a similarity measure often used in information retrieval. In general,given two n-dimensional vectors V and W, the cosine similarity computes the cosine of the angle $\alpha$ between
these two vectors as
$$CosineSimilarity(V,W) = cos(\alpha) = \frac{V \cdot W}{||V|| \cdot ||W||}$$

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer
stem = PorterStemmer()

from nltk.tokenize import word_tokenize

texts = ['cats and dogs like food',
          'cats like cat food',
          'dogs like dog food',
          'horses like dogs']
i=0
for text in texts:
    lem_words = []
    tokenized_word=word_tokenize(text)
    for word in tokenized_word:
        lem_words.append(lem.lemmatize(word,"v"))
        #print("Stemmed Word:",stem.stem(word))   
        #print(lem.lemmatize(words))
    texts[i] = " ".join(lem_words)
    i=i+1
texts


['cat and dog like food',
 'cat like cat food',
 'dog like dog food',
 'horse like dog']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer()
smat = cvec.fit_transform(texts)
import pandas as pd
import numpy as np
 
# make the DTM
dtm = pd.DataFrame(smat.toarray(), columns=cvec.get_feature_names())

dtm

,and,cat,dog,food,horse,like
0,1,1,1,1,0,1
1,0,2,0,1,0,1
2,0,0,2,1,0,1
3,0,0,1,0,1,1


In [ ]:
import sklearn

sklearn.metrics.pairwise.cosine_similarity(dtm)

array([[1.        , 0.73029674, 0.73029674, 0.51639778],
       [0.73029674, 1.        , 0.33333333, 0.23570226],
       [0.73029674, 0.33333333, 1.        , 0.70710678],
       [0.51639778, 0.23570226, 0.70710678, 1.        ]])

In [ ]:
#sklearn.metrics.pairwise.cosine_similarity(dtm, np.array([0,0,0,0,1,0]))

### EDIT-BASED SIMILARITY

We now focus on a second family of similarity measures,so called edit-based similarity measures.
In contrast to token-based measures, strings are considered as a whole and are not divided into sets
of tokens. However, to account for errors, such as typographical errors, word swaps and so on, edit-
based similarities allow different edit operations to transform one string into the other,e.g.,*insertion* of characters, character *swaps*, *deletion* of characters, or *replacement* of characters.

In [ ]:
import nltk
 
mistake = "ligting"
 
words = ['apple', 'bag', 'drawing', 'listing', 'linking', 'living', 'lighting', 'orange', 'walking', 'zoo']
 
for word in words:
    ed = nltk.edit_distance(mistake, word)
    print(word, ed)

apple 7
bag 6
drawing 4
listing 1
linking 2
living 2
lighting 1
orange 6
walking 4
zoo 7


In [ ]:
#correction('korrectud')

In [ ]:
import editdistance

In [ ]:
editdistance.eval('banana', 'bahama')

2

In [ ]:
import stringdist
stringdist.levenshtein('test', 'testing')

3

### Your own Spelling Corrector

Here are the four pillars of our spelling checker:

1. **Selection Mechanism**: argmax
    We choose the candidate with the highest combined probability.

2. **Candidate Model**: c ∈ candidates
    This tells us which candidate corrections, c, to consider.

3. **Language Model**: P(c)
    The probability that c appears as a word of English text. For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07.

4. **Error Model**: P(w|c)
    The probability that w would be typed in a text when the author meant c. For example, P(teh|the) is relatively high, but P(theeexyz|the) would be very low. 

### Candidates: 

We entertain all words within a constant stringdistance as candidates for the correctly spelled word.

First a new concept: a simple edit to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter). The function **edits1** returns a set of all the edited strings (whether words or not) that can be made with one simple edit: 

In [ ]:
import re


def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


This can be a big set. For a word of length n, there will be $n$ deletions, $n-1$ transpositions, $26n$ alterations, and $26(n+1)$ insertions, for a total of $54n+25$ (of which a few are typically duplicates). For example, 

In [ ]:
len(edits1('somthing'))

442

However, if we restrict ourselves to words that are known—that is, in the dictionary— then the set is much smaller.
Why not read in all Harry Potter novels and define them as our dictionary.

The function *words* breaks text into words, then the variable WORDS holds a Counter of how often each word appears, and *P* estimates the probability of each word, based on this Counter: 


In [ ]:
#os.system("cat data/HP/*.txt > big.txt")  

In [ ]:
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('data/HP/big.txt', encoding='latin-1').read()))

In [ ]:
def known(words): return set(w for w in words if w in WORDS)

known(edits1('somthing'))

{'something', 'soothing'}

We'll also consider corrections that require two simple edits. This generates a much bigger set of possibilities, but usually only a few of them are known words:

In [ ]:
def edits2(word): return ____

len(set(edits2('something'))

known(edits2('something'))

known(edits2('somthing'))

#### Language Model: P(c) 

In [ ]:
def P(word, N=sum(WORDS.values())): return WORDS[word] / N

In [ ]:
#WORDS.most_common(100)
P('looking')

0.0015779535814197303

#### Selection Mechanism: 

In Python, max with a key argument does 'argmax'. 

In [ ]:
max(WORDS,key=P)

'the'

#### Simplified Error model

Write a function *candidates(word)* that produces the first non-empty list of candidates in order of priority:

1. The original word, if it is known; otherwise
2. The list of known words at edit distance one away, if there are any; otherwise
3. The list of known words at edit distance two away, if there are any; otherwise
4. The original word, even though it is not known. 

In [ ]:
def correction(word): return max(candidates(word), key=P)

def candidates(word): 
    return 

In [ ]:
 correction('speling')              # insert
 correction('korrectud')            # replace 2
 correction('bycycle')                # replace
 correction('inconvient')       # insert 2
 correction('arrainged')            # delete
 correction('peotry')                  # transpose
 correction('peotryy')                # transpose + delete
 correction('word')                      # known

-----------------------------------------------------------
### Bayes Theorem

So far, the error model P(w|c) has been trivial: the smaller the edit distance, the smaller the error. 

A good spelling corrector would be much more sophisticated by relying on Bayes theorem.

We are trying to find the correction c, out of all possible candidate corrections, that maximizes the probability that c is the intended correction, given the original word w: 

$$
P(c | w) = \frac{P(c) P(w|c) }{P(w)} \sim P(c) P(w|c)
$$



Clearly we could use a better model of the cost of edits. We could use our intuition to assign lower costs for doubling letters and changing a vowel to another vowel (as compared to an arbitrary letter change), but it seems better to gather data: to get a corpus of spelling errors, and count how likely it is to make each insertion, deletion, or alteration, given the surrounding characters. We need a lot of data to do this well. If we want to look at the change of one character for another, given a window of two characters on each side, that's 266, which is over 300 million characters. You'd want several examples of each, on average, so we need at least a billion characters of correction data; probably safer with at least 10 billion. 